# <span style="color:blue;"> SISMID Module 9: Lab 3 </span>
    
## <span style="color:blue;"> Infection Spread on Networks </span>

#### <span style="color:blue;"> Shweta Bansal, Tom Hladish, Joel Miller

## Student name: [Callum Arnold]

#### Date: July 15, 2021 (Session 2D)



### Introduction

Today, we'll get some new tools under our belts for contact network epidemiology.
In particular, we will
* Work with contact networks
* Simulate disease spread over networks
* Use this tool to consider the difference between network structures

We'll be doing a number of things that we did in Lab 1, but by implementing them ourselves in Python
    
<img src="disease_network.jpg" width="700" />


### Outline:
1. Reading in the Data
2. Simulating Infection Spread on Networks
3. Structure &rarr; Function
    
##### As always, our modules are imported first. Notice that we have added a second part to each import statement. Here, we are just nicknaming the library so that we don't have to type out the whole word. For example, ``networkx`` is now called ``nx``

In [1]:
import networkx as nx           # Library for network functions
import numpy as np              # For mathetmatical functions
import random as rnd            # For random process functions
import matplotlib.pyplot as plt # For plotting functions

# Allow for network images to be displayed in notebook. This only needs to be done once in each program
%matplotlib inline

<br>

# 1. Reading in the data

Today, we're going to use a semi-empirical network for a small town, that was generated using a computational model using public data on households, schools, workpaces, hospitals etc in this town, as well heuristic knowledge about human social behavior.


#### <span style="color:orange;"> Ex 1.1: Load the network data by running the code below.

In [2]:
town_net = nx.read_edgelist("town_network_200.txt", delimiter = ",", nodetype = int)

#### <span style="color:orange;"> Ex 1.2: Get to know this town's contact network! Fill in the blanks: "This town has ___ individuals and on average, they contact ___ other individuals."
    
HINT: You'll need to use the ``<yournetworkname>.number_of_nodes`` and ``<yournetworkname>.number_of_edges`` functions.


In [3]:
town_net_nodes = town_net.number_of_nodes()
town_net_degree_vals = dict(town_net.degree).values()
town_net_avg_degree = sum(town_net_degree_vals) / len(town_net)

print("This town has", town_net_nodes, "individuals and on average, they contact", town_net_avg_degree, "other individuals.")

This town has 500 individuals and on average, they contact 4.968 other individuals.


# 2. Simulating Infection Spread on Networks

Now that we have a network ready to go, we are going to implement an algorithm to model disease transmission through this network: the *percolation model*. One approach to network epidemiology comes from physics, where researchers wanted to describe how a liquid can stochastically percolate through a semi-porous medium, like water percolating through sandstone. As it happens, this model can be readily adapted to describe how an infectious disease can "percolate" through a population. The percolation model can be worked out analytically, but today we will be working with it through simulation. To use a percolation model, we need to know two things: the contact network of our population, and the probability ($T$) that a contact will result in disease transmission, given contact between an infected and a susceptible individual. The probability of transmission ($T$), called *transmissibility*, is something that we generally try to estimate for a given disease based on data from real epidemics. The figure below illustrates the model/algorithm:

<center>
<img src = "percolation.png" width="700">
</center>


#### <span style="color:orange;"> Ex 2.1: Write down the algorithm as pseudocode for the percolation model.

1. Infect node i at time t0
2. For edge j of infectious node i:
   1. if node j is not I or R:
      1. infect with probability T
   2. else:
      1. stop

#### <span style="color:orange;"> Ex 2.2: Write a function (call it `percolation_on_network`) for the algorithm you specified above.

* HINT 1: Remember a definition for a function that takes two arguments has this format:
    
```Python
def myfunctionname(inputvariable1, inputvariable2):
    
    [Calculate output by doing things to input]
        
return output
```
    
* HINT 2: While there are other ways of doing this, I would suggest keeping track of susceptible/infected/recovered nodes using lists.
    
* HINT 3: You can use ``yourlist.append(itemtoappend)`` to add an item to a list and you can use ``yourlist.remove(itemtoremove)`` to remove an item from a list. Additionally, you can use ``if myitem in mylist:`` to figure out if an item is in a given list or ``if myitem not in myList`` to do the opposite.
    
* HINT 4: You can generate random numbers (uniform between 0 and 1) using ``rnd.random()``

In [4]:
I = []
R = []

def percolation_on_network(network, beta):
    init_node = np.random.randint(low = 1, high = network.number_of_nodes())
    I.append(init_node)

    while len(I) > 0:
        for infected in I:
            for neighbor in network.neighbors(infected):
                trans_prob = rnd.random()
                if ((trans_prob < beta) and (neighbor not in I) and (neighbor not in R)):
                    I.append(neighbor)
            I.remove(infected)
            R.append(infected)
    
    return(len(R))
        

#### <span style="color:orange;"> Ex 2.3: Next, call the above function to simulate disease spread on your network (`town_net`) with a transmissibility of 0.3. What does the return value from your function mean?
    
* HINT: Your function call has this format:
```Python
myanswer = myfunctionname(inputvalue1, inputvalue2)
```

In [5]:
percolation_on_network(network = town_net, beta = 0.3)

292

#### <span style="color:orange;"> Ex 2.4: Now, let's run the `percolation_on_network()` function a few more times (say 10 times). What happens and why?

* HINT: When I say run it multiple times, I mean call the function 10 times and output the result for each run.

In [6]:
import pandas as pd

In [11]:
results = {"Run": [], "Final Number Recovered": []}
for i in range(1, 11):
    recovered = percolation_on_network(network = town_net, beta = 0.3)
    results["Run"].append(i)
    results["Final Number Recovered"].append(recovered)

pd.DataFrame(results)

,Run,Final Number Recovered
0,1,374
1,2,375
2,3,381
3,4,382
4,5,383
5,6,386
6,7,392
7,8,393
8,9,395
9,10,396


#### You have just run a **Monte Carlo Simulation**, a term coined in the 1940's during the Manhattan project (nuclear weapons development project during WWII) and run on the ENIAC (one of the first computers in the world).

So, let's take the results of these indivdiual simulations and provide some interpretation. In particular, we want to:

1. Call the percolation_on_network() function
2. Get the returned number of infected individuals
3. Decide whether this number of infected indivdiuals is a large epidemic (say, bigger than 5% of the population) or a small outbreak (smaller than or equal to 5% of population size)
4. Based on the decision, save the result for number of infected indivdiuals in a large_epidemic list or in a small_outbreak list 
5. Repeat Steps 1 to 4 a large number of times (say, 250)
6. Report the average large_epidemic size, and the average small_outbreak size. Also report the probability_of_epidemic

#### <span style="color:orange;"> Ex 2.5: Write a new function (called percolation_results) that implements the above algorithm to interpret the results of your Monte Carlo simulations (as large epidemics and small outbreaks)

* HINT 1: While you're perfecting this function, I'd recommend using the print function to help you debug it

* HINT 2: While you're perfecting this function, I'd only run Step 5 a few times (say, 5) so that the function doesn't take forever to run. Then, once you have it working, you can set it up to run 250 times.

# 3. Structure &rarr; Function: the impact of network structure on disease spread

Let's investigate the impact of network structures on epidemic consequences (such as large epidemic size). To do this we will use two *idealized* networks, which we discussed in class. An idealized network isn't explicitly defined by empirical data, but instead is generated (using code) to have certain properties. We will focus on two specific idealized networks: an Erdos-Renyi network and a scale-free network (generated using the configuration model). Here are two visualizations of networks of these types:

<center>
<table>
<tr>
<td>
<img src = "erdos_renyi.png" width=300>
</td>
<td>
</td>
<td>
<img src = "scalefree.png" width=300>
</td>
</tr>
</table>
</center>

As you might suspect from the visualizations above, these two graphs have very different structure. The Erdos-Renyi network [left] is made up of individuals with approximately the same degree (number of edges or contacts). The scale-free network [right], on the other hand, is made up mostly of nodes with very low degree (1 or 2) and a few nodes with very high degree. The key to making these two networks comparable is that they both have the same average degree.

#### <span style="color:orange;"> Ex 3.1: Based on the above, what is different about the interactions between individuals (i.e. degrees) in these two networks? How do you think you could express this statistically? (Text answer only.)

#### <span style="color:orange;"> Ex 3.2: What is your hypothesis about the spread of infection in these two networks? Which do you expect to result in larger epidemics?

#### <span style="color:orange;"> Ex 3.3: Add code below to read these two networks from their edgelists: `network_er.txt` and `network_scalefree.txt`


#### <span style="color:orange;"> Ex 3.4: Using these two new networks, simulate the spread of a lowly transmissible disease like SARS (T = 0.1) through both populations.
* HINT 1: Use your ``percolation_results()`` function for this
    
* HINT 2: You may want to look at any/all output (more than just the average size) to get a clear idea of what is happening.

#### <span style="color:orange;"> Ex 3.5: Using these two new networks, simulate the spread of a highly transmissible disease like smallpox (T = 0.5) through both populations.

#### Now, let's look at what these results might look like across the whole range of transmissibility values.

<img src = "percolation_results.png" width=900>



#### <span style="color:orange;"> Ex 3.6: Provide an explanation of the results above. Explain the epidemic outcomes in terms of the structures of the two networks. Do they support your hypothesis above?
    
* HINT: Use the network visualizations above to gain some intuition